In [6]:
import numpy as np
A=np.array([[11,12,13],[21,22,23],[31,32,33]])
B=np.ones(shape=(3,3))

In [1]:
import tensorflow as tf


In [8]:
a=tf.constant([1,2],name="a")
b=tf.constant([2,4],name="b")
result = a+b
print(result)

#上面只是定义了计算图，并没有运行计算图，所以不会输出运算结果

sess=tf.Session()
a=sess.run(result)
print(a)
sess.close

#打开会话，运行计算图，关闭计算图

Tensor("add:0", shape=(2,), dtype=int32)
[3 6]


<bound method BaseSession.close of <tensorflow.python.client.session.Session object at 0x000001F5A9E4E4E0>>

In [9]:
with tf.Session() as sess:
    a=tf.constant([1,2,3,4])
    b=tf.constant([1,2,3,4])
    result=tf.multiply(a,b)
    c=sess.run(result)
    print(c)
    
print(c)

# with 结束，计算会话自动关闭

[ 1  4  9 16]
[ 1  4  9 16]


In [10]:
import tensorflow as tf
#（2，3，1单元）3层前向神经网络（无激活函数）

#生成服从标准差为1的正态分布的随机变量，作为初始化矩阵
w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

x=tf.constant([[0.7,0.9]])
#矩阵乘法
a=tf.matmul(x,w1)
b=tf.matmul(a,w2)

sess=tf.Session()
#需要运行初始化赋值，前面只是定义，没运算
sess.run(w1.initializer)
sess.run(w2.initializer)
y=sess.run(b)

sess.close()
print(y)

with tf.Session() as sess:
    w3=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
    w4=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))
    x1=tf.constant([[0.7,0.9]])
    a1=tf.matmul(x,w3)
    b1=tf.matmul(a1,w4)
    #可以直接嵌入初始化
    sess.run(tf.global_variables_initializer())
    print(sess.run(b1))


[[ 3.95757794]]
[[ 3.95757794]]


In [11]:
import tensorflow as tf
w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

#因为需要重复输入x，而每建一个x就会生成一个结点，计算图的效率会低。所以使用占位符
x=tf.placeholder(tf.float32,shape=(1,2))
x1=tf.placeholder(tf.float32,shape=(3,2))
a=tf.matmul(x,w1)
a1=tf.matmul(x1,w1)
y=tf.matmul(a,w2)
y1=tf.matmul(a1,w2)

sess=tf.Session()
sess.run(tf.global_variables_initializer())
#运行y时将占位符填上，feed_dict为字典，变量名不可变
y_hat=sess.run(y,feed_dict={x:[[0.7,0.9]]})
y_hat1=sess.run(y1,feed_dict={x1:[[0.7,0.9],[0.2,0.5],[1,2]]})# batch = 3
print(y_hat)
print(y_hat1)
sess.close


[[ 3.95757794]]
[[ 3.95757794]
 [ 1.657197  ]
 [ 7.20209646]]


<bound method BaseSession.close of <tensorflow.python.client.session.Session object at 0x000001F5A9F53F60>>

In [2]:
import tensorflow as tf
from numpy.random import RandomState

batch_size=10
w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

# None 可以根据batch 大小确定维度，在shape的一个维度上使用None，方便不大的batch
x=tf.placeholder(tf.float32,shape=(None,2))
y=tf.placeholder(tf.float32,shape=(None,1))

a=tf.matmul(x,w1)
yhat=tf.matmul(a,w2)

#定义交叉熵为损失函数，训练过程使用Adam算法最小化交叉熵
cross_entropy=-tf.reduce_mean(y*tf.log(tf.clip_by_value(yhat,1e-10,1.0)))
train_step=tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

rdm=RandomState(1)
data_size=516

#生成两个特征，共data_size个样本
X=rdm.rand(data_size,2)
#定义规则给出样本标签，所有x1+x2<1的样本认为是正样本，其他为负样本。Y，1为正样本
Y = [[int(x1+x2 < 1)] for (x1, x2) in X]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(w1))
    print(sess.run(w2))
    steps=11000
    for i in range(steps):
        
        #选定每一个批量读取的首尾位置，确保在1个epoch内采样训练
        start = i * batch_size % data_size
        end = min(start + batch_size,data_size)
        sess.run(train_step,feed_dict={x:X[start:end],y:Y[start:end]})
        if i % 1000 == 0:
            training_loss= sess.run(cross_entropy,feed_dict={x:X,y:Y})
            print("在迭代 %d 次后，训练损失为 %g"%(i,training_loss))

[[-0.81131822  1.48459876  0.06532937]
 [-2.4427042   0.0992484   0.59122431]]
[[-0.81131822]
 [ 1.48459876]
 [ 0.06532937]]
在迭代 0 次后，训练损失为 0.047106
在迭代 1000 次后，训练损失为 0.0114981
在迭代 2000 次后，训练损失为 0.00481489
在迭代 3000 次后，训练损失为 0.00285046
在迭代 4000 次后，训练损失为 0.00207789
在迭代 5000 次后，训练损失为 0.00158399
在迭代 6000 次后，训练损失为 0.0011365
在迭代 7000 次后，训练损失为 0.000963961
在迭代 8000 次后，训练损失为 0.000769849
在迭代 9000 次后，训练损失为 0.000558014
在迭代 10000 次后，训练损失为 0.000354686


#### 激活函数和偏置项：
a=tf.nn.relu(tf.matmul(x,w1)+biases1)

yhat=tf.nn.relu(tf.matmul(a,w2)+biases2)

#### 交叉熵函数
cross_entropy=-tf.reduce_mean(y*tf.log(tf.clip_by_value(yhat,1e-10,1.0)))

tf.reduce_mean(x)表示计算全局平均值。tf.clip_by_value()函数可以将张量中的数值限制在一个范围内。tf.log()对张量内的所有元素依次求对数。交叉熵函数一般会与softmax回归一起使用，TensorFlow将它们进行了统一封装：cross_entropy=tf.nn.softmax_cross_entropy_with_logits(yhat,y)



#### 均方误差损失函数
mse=tf.reduce_mean(tf.square(y-yhat))

tf.select(tf.greater(y,yhat),y-yhat,yhat-y)

tf.greater()的输入是两个张量，比较两个张量中的每一个元素，并返回比较结果（true或false的向量）。tf.select()有三个参数，第一个参数条件为真时选择第二个参数中的值，否则选择第三个参数的值。

In [10]:
#decayed_learning_rate=learning_rate*decay_rate^(global_steps/decay_steps)，指数衰减函数的定义

global_step=tf.Variable(0)
#使用exponential_decay生成学习速率，因为staircase=tire，每100次迭代，学习率×0.96
learning_rate=tf.train.exponential_decay(0.1,global_step,100,0.96,staircase=True)
#在minimize中导入global_step将自动更新
#learning_step=tf.train.GtadientDescentOptimizer(learning_rate).minimize(loss_function,global_step=global_step)


#### 带L2正则化的损失函数
w=tf.Variable(tf.random_normal([2,1],stddev=1,seed=1))

yhat=tf.matmul(x,w)

loss=tf.reduce_mean(tf.square(y-yhat))+tf.contrib.layers.l2_regularizer(lambda)(w)

In [1]:
import tensorflow as tf
w=tf.constant([[1.0,-5.0],[-3.0,4.0]])
with tf.Session() as sess:
    #L1正则化：(1+5+3+4)×0.5
    print(sess.run(tf.contrib.layers.l1_regularizer(0.5)(w)))
    #L2正则化：（1+25+9+16）/2×0.5，L2正则化会处以2，无偏估计？
    print(sess.run(tf.contrib.layers.l2_regularizer(0.5)(w)))

6.5
12.75


神经网络结构复杂后，定义网络结构的部分和计算损失函数的部分可能不在同一个函数中。所以采用collection在一个计算图中保留一组实体（如张量）。

In [2]:
import tensorflow as tf

#通过集合（collection）计算一个5层神经网络带L2正则化的损失函数

#随机正态初始化一层神经网络的权重，并将权重的L2正则化损失加入名为losses的集合中，返回初始化的权重
def get_weight(shape,lambd):
    var=tf.Variable(tf.random_normal(shape),dtype=tf.float32)
    
    #tf.add_to_collection函数将新生成变量的L2正则化损失项加入集合，第一个参数为集合名，第二个参数为加入集合的内容。
    tf.add_to_collection('losses',tf.contrib.layers.l2_regularizer(lambd)(var))
    return var

x=tf.placeholder(tf.float32,shape=(None,2))
y=tf.placeholder(tf.float32,shape=(None,1))
batch_size=8

#定义每一层中结点个数和层数
layer_dimension=[2,10,10,10,1]
n_layers=len(layer_dimension)

#该变量维护前向传播时最深层的结点，最开始为输入层
cur_layer=x

#输入层结点个数
in_dimension=layer_dimension[0]

#通过循环生成5层全连接神经网络
for i in range(1,n_layers):
    
    #下一层节点数
    out_dimension=layer_dimension[i]
    
    #生成当前层中权重的变量，并将这个变量的L2正则化损失加入计算图上的集合
    #[in_dimension,out_dimension],例第一层到第二层之间的权重维度为2×10
    weight=get_weight([in_dimension,out_dimension],0.001)
    
    #偏置项和后一层维度相等，为什么是wx+0.1，而不是wx+b？？
    bias=tf.Variable(tf.constant(0.1,shape=[out_dimension]))
    
    #使用ReLU激活函数，cur_layer储存传播一层后的激活情况，后一层激活函数的输出
    cur_layer=tf.nn.relu(tf.matmul(cur_layer,weight)+bias)
    in_dimension=layer_dimension[i]
    
mse_loss=tf.reduce_mean(tf.square(y-cur_layer))

#将均方误差函数加入损失集合
tf.add_to_collection('losses',mse_loss)

#get_collection返回一个列表，这个列表是集合中的所有元素，这些元素就是组成损失函数的误差和正则项，相加得最终损失函数
loss=tf.add_n(tf.get_collection('losses'))



In [4]:
import tensorflow as tf

#tf.assign(A, new_number),这个函数的功能主要是把A的值变为new_number
A=tf.Variable(tf.constant(0.0),dtype=tf.float32)
with tf.Session() as sess:  
    sess.run(tf.global_variables_initializer())  
    print (sess.run(A))  
    sess.run(tf.assign(A, 10))  
    print (sess.run(A))  

0.0
10.0


In [2]:
import tensorflow as tf

#滑动平均模型

#定义一个变量计算滑动平均，初始值为0，所有需要计算滑动平均的变量必须是实数型
v1=tf.Variable(0,dtype=tf.float32)

#step变量模拟神经网络中的迭代次数，用于动态控制衰减率
step=tf.Variable(0,trainable=False)

#定义一个滑动平均的类，初始化时给定了衰减率和控制衰减率的变量step
ema=tf.train.ExponentialMovingAverage(0.99,step)

#定义一个更新变量的滑动平均操作，给定一个列表，每次执行操作时更新列表所有变量
maintain_averages_op=ema.apply([v1])

with tf.Session() as sess:
    
    #初始化所有变量
    sess.run(tf.global_variables_initializer())
    
    #通过ema.average(v1)获取滑动平均之后变量的取值。初始化后，v1的值和滑动平均都为0
    print(sess.run([v1,ema.average(v1)]))
    
    #更新变量v1的值为5,tf.assign将数值分配给变量
    sess.run(tf.assign(v1,5))
    
    #更新v1的滑动平均值。衰减率为min{0.99,(1+step)/(10+step)≈0.1}=0.1，所以v1的滑动平均值会更新为 0.1×0+0.9×5=4.5
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)]))
    
    #将迭代设置为10000步
    sess.run(tf.assign(step,10000))
    
    #更新v1的值为10
    sess.run(tf.assign(v1,10))
    
    #更新v1的滑动平均值。衰减率为min{0.99,(1+step)/(10+step)≈0.999}=0.99，所以v1的滑动平均会被更新为0.99×4.5+0.01×10=4.555
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)]))
    
    #再次更新滑动平均值，得到新的滑动平均值为0.99×4.555+0.01×10=4.60945
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)]))



[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.5549998]
[10.0, 4.6094499]


### MNIST 手写字体识别

In [3]:
#导入数据
import tensorflow as tf

#原网站提供了6W张训练图片和1W张测试图片，导入的该工具会从训练图片分出5000张作为验证集
from tensorflow.examples.tutorials.mnist import input_data

#读取路径为当前路径下的data文件夹下的MNIST文件夹内，如果该文件夹没有，则自动下载数据至该文件夹
mnist = input_data.read_data_sets("./data/MNIST/", one_hot=True)

print("Training data size: ", mnist.train.num_examples) 
print ("Validating data size: ", mnist.validation.num_examples) 
print ("Testing data size: ", mnist.test.num_examples) 

Extracting ./data/MNIST/train-images-idx3-ubyte.gz
Extracting ./data/MNIST/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST/t10k-labels-idx1-ubyte.gz
Training data size:  55000
Validating data size:  5000
Testing data size:  10000


In [4]:
#为了方便使用SGD，mnist.train.next_batch函数可以从所有训练数据中取一个小批量投入训练

batch_size=100

#从训练集选取batch_size个训练数据
xs,ys=mnist.train.next_batch(batch_size)

#将图片展开成一个长度为28×28=784的一维数组，一张图片可作为一个特征向量。所以batch为100的矩阵维度为100×784
print('X shapr:',xs.shape)
print('Y shape:',ys.shape)

X shapr: (100, 784)
Y shape: (100, 10)


In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/MNIST/", one_hot=True)


#输入结点数为像素点数，输出结点数为类别数
INPUT_NODE=784
OUTPUT_NODE=10

#一个隐藏层
LAYER1_NODE=500

#一个批量中的样本量，数据量越小训练过程越接近SGD，数据量越大训练过程越接近梯度下降
BATCH_SIZE=100

#学习率和学习衰减率
LEARNING_RATE_BASE=0.8
LEARNING_RATE_DECAY=0.99

#正则化系数、迭代次数和滑动平均衰减率
REGULARIZATION_RATE=0.0001
TRAINING_STEPS=3000
MOVING_AVERAGE_DECAY=0.99

#定义推断函数，给定所有参数下计算神经网络的前向传播结果。参数avg_class可确定推断中使不使用滑动平均模型
def inference(input_tensor,avg_class,weights1,biases1,weights2,biases2):
    
    #没有提供滑动平均类时，直接使用参数当前的取值
    if avg_class == None:
        
        #计算隐藏层前向传播结果，使用ReLU激活函数
        layer1=tf.nn.relu(tf.matmul(input_tensor,weights1)+biases1)
        
        #计算输出层的前向传播结果
        return tf.matmul(layer1,weights2)+biases2
    else:
        
        #首先使用avg_class.averaage函数计算变量的滑动均值，然后计算相应的前向传播结果
        layer1=tf.nn.relu(tf.matmul(input_tensor,avg_class.average(weights1))+avg_class.average(biases1))
        return tf.matmul(layer1,avg_class.average(weights2))+avg_class.average(biases2)
    
#模型训练函数

def train(mnist):
    x=tf.placeholder(tf.float32,[None,INPUT_NODE],name='x-input')
    y=tf.placeholder(tf.float32,[None,OUTPUT_NODE],name='y-input')
    
    #生成隐藏层参数
    weights1=tf.Variable(tf.truncated_normal([INPUT_NODE,LAYER1_NODE],stddev=0.1))
    biases1=tf.Variable(tf.constant(0.1,shape=[LAYER1_NODE]))
    
    #生成输出层参数
    weights2=tf.Variable(tf.truncated_normal([LAYER1_NODE,OUTPUT_NODE],stddev=0.1))
    biases2=tf.Variable(tf.constant(0.1,shape=[OUTPUT_NODE]))
    
    #计算当前参数下前向传播的结果，这里设为‘None’不会计算滑动平均值
    y_hat=inference(x,None,weights1,biases1,weights2,biases2)

    #定义储存迭代数的变量，这个变量不需要计算滑动平均值，所以这里指定的这个变量为不饿训练变量（trainable=False）
    global_step=tf.Variable(0,trainable=False)
    
    #给定滑动平均衰减率和迭代数，初始化滑动平均类。
    variable_averages=tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    
    #在所有代表神经网络参数的变量上使用滑动平均，其他超参数不需要。tf.trainable_variables返回的就是图上的集合GraphKeys.TRAINABLE_VARIABLES中的元素。
    variables_averages_op=variable_averages.apply(tf.trainable_variables())
    
    #计算使用滑动平均后的前向传播结果，滑动平均不会改变变量本身，而是使用影子变量记录滑动平均值，需要使用滑动平均再明确调用average函数
    average_y_hat=inference(x,variable_averages,weights1,biases1,weights2,biases2)
    
    #~使用tf.argmax函数得到正确答案对应的类别编号
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_hat, labels=tf.argmax(y, 1))
    
    #计算当前批量中所有样本的交叉熵均值
    cross_entropy_mean=tf.reduce_mean(cross_entropy)
    
    #计算L2正则化损失函数
    regularizer=tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    
    #计算模型的正则化损失，只计算神经网络权重的正则化损失，不使用偏置项
    regularization=regularizer(weights1)+regularizer(weights2)
    
    #总损失函数
    loss=cross_entropy_mean+regularization
    
    #设置指数衰减学习率.基础学习率、当前迭代次数、一个epoch所需要的迭代次数、学习衰减率
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE,
        LEARNING_RATE_DECAY,
        staircase=True)
    
    #使用梯度下降优化算法优化损失函数，损失函数包括交叉熵损失和L2正则化损失
    train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
    
    #在训练神经网络模型时，每过一遍数据既需要通过反向传播来更新参数，也要更新每个参数的滑动平均值。为了一次完成多个操作
    #train_op=tf.group(train_step,variables_averages_op)
    with tf.control_dependencies([train_step,variables_averages_op]):
        train_op=tf.no_op(name='train')
    
    
    #检验使用滑动平均模型的前向传播结果是否正确。tf.argmax(average_y_hat,1)计算每一个样本的预测。average_y_hat是一个batch_size×10的二维数组，
    #每一行表示一个样本的前向传播结果。tf.argmax的第二个参数“1”表示选取最大值的操作只在第一个维度中进行，即每一行选取最大值的下标。
    #于是得到的结果为长度为batch的一维数组，数组的值表示每个样本对应识别的类别。tf.equal判断两个张量的每一维度是否相等，相等返回Ture。
    correct_prediction=tf.equal(tf.argmax(average_y_hat,1),tf.argmax(y,1))
    
    #该运算首先将布尔型数值转为实数型，再计算均值。该均值为模型在这一组数据上的正确率。
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

    #初始化会话并开始训练过程
    with tf.Session() as sess:
        
        tf.global_variables_initializer().run()
        
        #准备验证数据，可通过验证数据简要判断停止条件和训练效果。
        validate_feed = {x: mnist.validation.images, y: mnist.validation.labels}
        
        #准备测试数据，作为模型最终性能的判别标准
        test_feed={x:mnist.test.images,y:mnist.test.labels}
    
        # 迭代地训练神经网络。
        for i in range(TRAINING_STEPS):
            
            #每1000次输出1次验证集上的测试结果
            if i % 500 == 0:
                
                #计算滑动平均模型在验证集上的结果，数据集小可一次处理所有验证数据。如果验证集大，需要分为更小的batch。
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step(s), validation accuracy using average model is %g " % (i, validate_acc))
            
            #生成一次迭代需要用到的批量数据，并运行训练过程
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op,feed_dict={x:xs,y:ys})
        
        #训练结束后再测试集上计算最终模型准确度
        test_acc=sess.run(accuracy,feed_dict=test_feed)
        print(("After %d training step(s), test accuracy using average model is %g" %(TRAINING_STEPS, test_acc)))

    
    

Extracting ./data/MNIST/train-images-idx3-ubyte.gz
Extracting ./data/MNIST/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST/t10k-labels-idx1-ubyte.gz


In [4]:
train(mnist)

After 0 training step(s), validation accuracy using average model is 0.1354 
After 500 training step(s), validation accuracy using average model is 0.9682 
After 1000 training step(s), validation accuracy using average model is 0.975 
After 1500 training step(s), validation accuracy using average model is 0.9794 
After 2000 training step(s), validation accuracy using average model is 0.9816 
After 2500 training step(s), validation accuracy using average model is 0.983 
After 3000 training step(s), test accuracy using average model is 0.9815


### 变量管理
在定义inference函数时需要提供NN的所有参数，NN过大时馈送参数很麻烦。TensorFlow 提供了通过变量名称来创建或获取一个变量的机制。通过这个机制可以在不同函数中直接通过变量的名字来使用变量，而不需要将变量以参数的形式到处传递。TensorFlow主要通过tf.get_variable和tf.variable_scope函数实现通过变量名获取变量值。

In [2]:
#下面两个定义等价
v1=tf.Variable(tf.constant(1.0,shape=[1]),name='v')

#提供未读信息与初始化方法，指定变量名称为必须项
v=tf.get_variable("v",shape=[1],initializer=tf.constant_initializer(1.0))


tf.get_variable首先会创建名为“v”的参数，有同名参数会创建失败。tf.get_variable获取一个已创建的变量需要使用tf.variable_scope来控制。

In [3]:
#在名为foo的命名空间内创建名为v的变量
with tf.variable_scope("foo"):
    v=tf.get_variable('v',[1],initializer=tf.constant_initializer(1.0))
    
#因为在命名空间foo中已经存在名字为v的变量，下面的代码将报错:
#with tf.variable_scope("foo"):
#    v=tf.get_variable("v",[1])

#在生成上下文管理器时，将参数reuse设置为Ture。这样tf.get_variable函数将直接获取已经声明的变量
with tf.variable_scope("foo",reuse=True):
    v1=tf.get_variable('v',[1])
    print (v == v1)
    
#将参数reuse设置为ture时，tf.variable_scope将只能获取已经创建的变量。若命名空间foo无该变量，则会报错。

True


In [8]:
#当tf.variable_scope函数嵌套时，reuse参数的取值如何确定

with tf.variable_scope('root'):
    
    #可以通过tf.get_variable_scope().reuse函数获取当前上下文管理器中reuse参数的信息,以下输出False，即最外层reuse是False
    print(tf.get_variable_scope().reuse)
    
    #新建一个嵌套的上下文管理器，并指定reuse为Ture
    with tf.variable_scope('foo',reuse=True):
        #输出True
        print(tf.get_variable_scope().reuse)
        
        #新建一个嵌套的上下文管理器，但不指定reuse，这时reuse的取值会和外面一层保持一致，以下输出True
        with tf.variable_scope('bat'):
            print(tf.get_variable_scope().reuse)
    #退出reuse设置为True的上下文后，又会输出False
    

False
True
True


tf.variable_scope函数生成上下文管理器，也会创建一个TensorFlow中的命名空间，在命名空间中创建的变量名称会带上命名空间名作为前缀。以下展示了如何通过tf.variable_scope管理变量的名称。

In [9]:
v1=tf.get_variable('v',[1])
#“:0”表示这个变量是生成变量这个运算的第一个结果
print(v1.name)

with tf.variable_scope('foo'):
    v2=tf.get_variable('v',[1])
    #在tf.variable_scope中创建的变量，名称前会加入命名空间的名称
    print(v2.name)
    
with tf.variable_scope('foo'):
    with tf.variable_scope('bar'):
        v3=tf.get_variable('v',[1])
        print(v3.name)
        
#创建一个名称为空的命名空间，并设置reuse=True
with tf.variable_scope('',reuse=True):
    #可以直接通过带命名空间名称的变量名来获取其他命名空间下的变量
    v4=tf.get_variable('foo/bar/v',[1])
    print(v4.name)
    print(v4 == v3)

v:0
foo/v:0
foo/bar/v:0
foo/bar/v:0
True


通过变量管理，我们可以将上面定义的推断过程函数做一些改进。

In [2]:
INPUT_NODE=10
LAYER1_NODE=32
OUTPUT_NODE=2

def inference (input_tensor,reuse=False):
    
    #定义第一层神经网络的变量和前向传播过程
    #根据传进来的reuse判断是创建新变量还是使用已经创建好的。没有传入reuse参数时，默认为False.
    with tf.variable_scope('layer1',reuse=reuse):
        weights=tf.get_variable('weights',[INPUT_NODE,LAYER1_NODE],initializer=tf.truncated_normal_initializer(stddev=0.1))
        biases=tf.get_variable('biases',[LAYER1_NODE],initializer=tf.constant_initializer(0.0))
        layer1=tf.nn.relu(tf.matmul(input_tensor,weights)+biases)
        
    #定义第二层神经网络的变量与前向传播过程，matmul函数注意左乘和右乘的区别
    with tf.variable_scope('layer2',reuse=reuse):
        weights=tf.get_variable('weights',[LAYER1_NODE,OUTPUT_NODE],initializer=tf.truncated_normal_initializer(stddev=0.1))
        biases=tf.get_variable('biases',[OUTPUT_NODE],initializer=tf.constant_initializer(0.0))
        layer2=tf.matmul(layer1,weights)+biases
    return(layer2)

x=tf.placeholder(tf.float32,[None,INPUT_NODE])
y=inference(x)

#使用训练好的神经网络进行推断时，直接调用inference(new_x,True)。使用上述代码表述1，不需要将所有变量作为参数传递到不同函数中。

#### Tensorflow代码持久化
保存训练好的模型

In [5]:
#保存TensorFlow计算图
import tensorflow as tf

#声明两个变量，并计算它们的和
v1=tf.Variable(tf.constant(1.0,shape=[1]),name='v1')
v2=tf.Variable(tf.constant(2.0,shape=[1]),name='v2')
result=v1+v2

#声明tf.train.Saver()类用于保存模型
saver=tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.save(sess,'model/1/model1.ckpt')

#*.ckpt.meta保存了Tensorflow计算图的结构。*.ckpt保存了TensorFlow中每个变量的取值。checkpoint保存了一个目录下所有的模型文件列表

In [1]:
import tensorflow as tf

v1=tf.Variable(tf.constant(22.0,shape=[1]),name='v1')
v2=tf.Variable(tf.constant(2.0,shape=[1]),name='v2')
result=v1+v2

saver=tf.train.Saver()

with tf.Session() as sess:
    
    #加载已保存的模型，并从已保存模型中的变量值计算加法
    saver.restore(sess,'model/1/model1.ckpt')
    print(sess.run(result))
    
#模型计算图不能更改，更改了就不能运行，只少了初始化变量，变量值从保存中抽取

INFO:tensorflow:Restoring parameters from model/1/model1.ckpt
[ 3.]


以上默认保存和加载TensorFlow计算图上定义的全部变量。如需要选择保存或加载部分变量，在声明tf.train.Saver类时可以提供一个列表指定需要保存或加载的变量。例如saver=tf.train.Saver([v1])构建声明，那么只会保存或加载变量v1。tf.train.Saver也支持在保存或加载时给变量命名。

In [2]:
#这里声明的变量名称和已保存模型中的变量名称不同
v_1=tf.Variable(tf.constant(3.0,shape=[1]),name='v_1')
v_2=tf.Variable(tf.constant(2.0,shape=[1]),name='v_2')
result=v_1+v_2

#如果直接使用tf.train.Saver()加载模型会找不到变量而报错
#使用字典重命名变量就能加载原来的模型。

saver=tf.train.Saver({'v1':v_1,'v2':v_2})
with tf.Session() as sess:
    
    saver.restore(sess,'model/1/model1.ckpt')
    print(sess.run(result))

INFO:tensorflow:Restoring parameters from model/1/model1.ckpt
[ 3.]


In [1]:
#保存滑动平均模型的案例

import tensorflow as tf

v=tf.Variable(0,dtype=tf.float32,name='v')

#在没有声明滑动平均模型时只有一个变量v，下面语句会输出“v:0”
for variables in tf.global_variables():
    print(variables.name)
    
ema=tf.train.ExponentialMovingAverage(0.99)
maintain_averages_op=ema.apply(tf.all_variables())

#在声明滑动平均模型后，TensorFlow会自动生成一个影子变量
for variables in tf.global_variables():
    print(variables.name)
    
saver=tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.assign(v,10))
    sess.run(maintain_averages_op)
    
    #保存时，tf会将v和v/ExponentialMovingAverage两个变量都存下来
    saver.save(sess,'model/test/movingaverage/ema.ckpt')
    print(sess.run([v, ema.average(v)]))

v:0
Instructions for updating:
Please use tf.global_variables instead.
v:0
v/ExponentialMovingAverage:0
[10.0, 0.099999905]


In [2]:
#以下展示通过重命名直接读取变量的滑动平均值
import tensorflow as tf
v=tf.Variable(0, dtype=tf.float32,name='v')

#通过变量重命名将原来变量v的滑动平均值直接赋值给v
saver=tf.train.Saver({'v/ExponentialMovingAverage':v})
with tf.Session() as sess:
    saver.restore(sess, 'model/test/movingaverage/ema.ckpt')
    print(sess.run(v))

INFO:tensorflow:Restoring parameters from model/test/movingaverage/ema.ckpt
0.0999999


In [1]:
#为了方便加载时重命名滑动平均值，tf.train.ExponentialMovingAverage提供了
#variables_to_restore函数生成tf.train.Saver所需要的变量重命名字典。

import tensorflow as tf

v=tf.Variable(0, dtype=tf.float32,name='v')
ema=tf.train.ExponentialMovingAverage(0.99)

#通过variables_to_restore函数可以直接生成上面代码中提供的字典{'v/ExponentialMovingAverage':v}
print(ema.variables_to_restore())

saver=tf.train.Saver(ema.variables_to_restore())
with tf.Session() as sess:
    saver.restore(sess,'model/test/movingaverage/ema.ckpt')
    print(sess.run(v))


{'v/ExponentialMovingAverage': <tf.Variable 'v:0' shape=() dtype=float32_ref>}
INFO:tensorflow:Restoring parameters from model/test/movingaverage/ema.ckpt
0.0999999


tf提供了convert_to_constants函数，该函数可以将计算图中的变量及其取值通过常量的方式保存，整个计算图和变量可以统一放在一个文件中。

In [3]:
import tensorflow as tf
from tensorflow.python.framework import graph_util

v1=tf.Variable(tf.constant(2.0, shape=[1]),name='v1')
v2=tf.Variable(tf.constant(8.0, shape=[1]),name='v2')
result = v1 + v2

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    #导出当前计算图的GraphDef部分，只需要这一部分就能完成从输入层到输出层的计算过程
    graph_def=tf.get_default_graph().as_graph_def()
    
    #将图中的变量及其取值转化为常量，同时将图中不必要的结点去掉。一些系统运算也会被转化为计算图中的结点（比如变量初始化操作）。
    #如果只关心程序中定义的某些计算，那么和这些计算无关的结点就没有必要导出并保存。
    #以下最后一个参数['add']给出了需要保存的结点名称。add结点是上面定义的求幂运算。
    output_graph_def=graph_util.convert_variables_to_constants(sess, graph_def,['add'] )
    
    #将导出的模型存入文件
    with tf.gfile.GFile('./model/test/1/combined_model.pb','wb') as f:
        f.write(output_graph_def.SerializeToString())

INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.


In [7]:
#以下可以直接计算定义的加法运算结果，当只需要计算图某个结点的取值时，该方法很方便

import tensorflow as tf
from tensorflow.python.platform import gfile

with tf.Session() as sess:
    model_filename='./model/test/1/combined_model.pb'
    
    #读取保存的模型文件，并将文件解析成对应的GraphDef Protocol Buffer。
    with gfile.FastGFile(model_filename,'rb') as f:
        graph_def=tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    #将graph_def中保存的图加载到当前的图中。return_elements=['add:0']给出了返回的张量名称。在保存的时候给出的是计算结点的名称，
    #所以为“add”。在加载的时候给出的张量的名称，所以是add:0。
    result=tf.import_graph_def(graph_def, return_elements=['add:0'])
    print(sess.run(result))

[array([ 10.], dtype=float32)]


### 重构MNIST识别
切分功能模块，推断过程抽象为单独库函数。模型分为三个模块：mnist_inference.py定义了前向传播过程与神经网络中的参数。
mnist_train.py定义了神经网络的训练过程。mnist_eval.py定义了测试过程

In [1]:
#mnist_inference.py
# -*- coding: utf-8 -*-
import tensorflow as tf

#定义神经网络结构相关的参数
INPUT_NODE=784
OUTPUT_NODE=10
LAYER1_NODE=500

#通过tf.get_variable函数获取变量。在训练神经网络时会创建这些变量；在测试时会通过保存的模型加载这些变量的取值。
#而且更加方便的是，因为可以在变量加载时将滑动平均变量重命名，所以可以直接通过同样的名字在训练是使用变量自身，
#而在测试时使用变量的滑动平均值。在这个函数中也会将变量的正则化损失加入损失集合。
def get_weight_variable(shape, regularizer):
    weights=tf.get_variable('weights',shape,initializer=tf.truncated_normal_initializer(stddev=0.1))
    
    #当给出了正则化生成函数时，将当前变量的正则化损失加入名为losses的集合。
    if regularizer != None:
        tf.add_to_collection('losses',regularizer(weights))
    return weights

#定义神经网络的前向传播过程
def inference(input_tensor,regularizer):
    
    #声明第一层神经网络的的变量并完成前向传播过程
    with tf.variable_scope('layer1'):
        #这里tf.get_variable或tf.Variable没有本质区别，因为在训练或测试中没有在同一个程序中多次调用该函数。
        #如果在同一程序中多次调用，在第一次调用后需要将reuse参数设为True。
        weights=get_weight_variable([INPUT_NODE, LAYER1_NODE], regularizer)
        biases=tf.get_variable('biases', [LAYER1_NODE], initializer = tf.constant_initializer(0.0))
        layer1=tf.nn.relu(tf.matmul(input_tensor, weights)+biases)
    
    #类似的声明第二层神经网络的变量并完成前向传播过程
    with tf.variable_scope('layer2'):
        weights=get_weight_variable([LAYER1_NODE, OUTPUT_NODE], regularizer)
        biases=tf.get_variable('biases', [OUTPUT_NODE], initializer = tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1, weights) + biases
    
    return layer2
    

In [ ]:
#mnist_train.py
# -*- coding: utf-8 -*-
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import mnist_inference

# 当前py文件在同一个地址

# 配置神经网络的参数
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 10000
MOVING_AVERAGE_DECAY = 0.99
MODEL_SAVE_PATH = "./model/fcn_mnist"
MODEL_NAME = "fcn_mnist.ckpt"


def train(mnist):
    x = tf.placeholder(tf.float32, [None, mnist_inference.INPUT_NODE], name='x-input')
    y = tf.placeholder(tf.float32, [None, mnist_inference.OUTPUT_NODE], name='y-output')

    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    # 调用推断过程
    y_hat = mnist_inference.inference(x, regularizer)
    global_step = tf.Variable(0, trainable=False)

    # 定义损失函数、学习率、滑动平均操作及训练过程
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_average_op = variable_averages.apply(tf.trainable_variables())
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_hat, labels=tf.argmax(y, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, global_step, mnist.train.num_examples / BATCH_SIZE,
                                              LEARNING_RATE_DECAY)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    with tf.control_dependencies([train_step, variables_average_op]):
        train_op = tf.no_op(name='train')

    # 初始化TF持久化类
    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        # 在训练过程中不再测试模型在验证数据上的表现，验证和测试的过程会有独立的过程完成
        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y: ys})

            # 每1000次迭代保存一次模型
            if i % 1000 == 0:
                # 输出模型在当前训练批量下的损失函数大小
                print('After %d training steps, loss on training batch is %g.' % (step, loss_value))

                # 保存当前模型，并使用global_step 参数特定地命名
                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)


def main(argv=None):
    mnist = input_data.read_data_sets('./data/MNIST/', one_hot=True)
    train(mnist)


if __name__ == '__main__':
    tf.app.run()


Extracting ./data/MNIST/train-images-idx3-ubyte.gz
Extracting ./data/MNIST/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST/t10k-labels-idx1-ubyte.gz
After 1 training steps, loss on training batch is 2.81242.


In [ ]:
#mnist_eval.py
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import mnist_inference
import mnist_train

#每10秒加载一次最新的模型，并在测试数据上测试最新模型的准确率
EVAL_INTERVAL_SECS=10

def evaluate(mnist):
    with tf.Graph().as_default() as g:
        #定义输入输出格式
        x=tf.placeholder(tf.float32, [None, mnist_inference.INPUT_NODE], name='x-input')
        y=tf.placeholder(tf.float32, [None, mnist_inference.OUTPUT_NODE], name='y-output')
        validate_feed={x:mnist.validation.images, y:mnist.validation.labes}
        
        #直接通过调用封装好的函数来计算前向传播结果。因为测试时不关注正则化损失值，所以用于计算正则化损失的函数可以设置为None
        y_hat=mnist_inference.inference(x, None)
        
        #使用前向传播结果计算准确度，如需对未知样本进行分类，使用tf.argmax(y_hat, 1)就可以得到输入样本的预测类别
        corret_prediction=tf.equal(tf.argmax(y_hat, 1), tf.argmax(y, 1))
        accuracy=tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        #通过变量重命名的方式加载模型，因此前向传播过程不需调用滑动平均操作取平均值。完全共用mnist_inference.py定义的传播过程
        variable_averages=tf.train.ExponentialMovingAverage(mnist_train.MOVING_AVERAGE_DECAY)
        variables_to_restore=variable_averages.variables_to_restore()
        
        #variables_to_restore函数生成tf.train.Saver类所需要的变量重命名字典
        saver=tf.train.Saver(variables_to_restore)
        
        #每隔EVAL_INTERVAL_SECS秒调用一次计算准确度的过程以检测训练过程中的正确率变化
        while True:
            with tf.Session() as sess:
                #tf.train.get_checkpoint_state函数会通过checkpoint文件自动找到目录中最新模型的文件名
                ckpt=tf.train.get_checkpoint_state(mnist_train.MODEL_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
                    #加载模型
                    saver.restore(sess, ckpt.model_checkpoint_path)
                    #通过文件名得到模型保存时迭代的次数
                    global_step=ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    accuracy_score=sess.run(accuracy, feed_dict=validate_feed)
                    print("After %s training step(s), validation accuracy = %g" % (global_step, accuracy_score))
                else:
                    print('No checkpoint file found')
                    return
            time.sleep(EVAL_INTERVAL_SECS)
            
def main(argv=None):
    mnist=input_data.read_data_sets('./data/MNIST/', one_hot=True)
    evaluate(mnist)
    
if __name__=='__main__':
    tf.app.run()
                    
        